In [5]:
import csv

input_path = "training_data/dolly-15k-compressions-gpt-test.csv"
output_path = "training_data/dolly-summarization-data.csv"

unique_prompts = set()
cleaned_rows = []

with open(input_path, "r", encoding="utf-8") as infile:
    reader = csv.reader(infile)
    header = next(reader)
    for row in reader:
        prompt = row[0]
        # Only keep if fifth column is not empty and prompt is unique
        if row[4].strip() and prompt not in unique_prompts:
            cleaned_rows.append(row)
            unique_prompts.add(prompt)

with open(output_path, "w", encoding="utf-8", newline="") as outfile:
    writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)
    writer.writerow(header)
    writer.writerows(cleaned_rows)

print(f"Cleaned data written to {output_path}. {len(cleaned_rows)} unique, complete rows retained.")

Cleaned data written to training_data/dolly-summarization-data.csv. 14779 unique, complete rows retained.


In [8]:
import csv

input_path = "training_data/dolly-summarization-data.csv"

total_original_tokens = 0
total_compression_tokens = 0
row_count = 0

with open(input_path, "r", encoding="utf-8") as infile:
    reader = csv.reader(infile)
    header = next(reader)
    for row in reader:
        try:
            orig_tokens = int(row[2])
            comp_tokens = int(row[3])
            total_original_tokens += orig_tokens
            total_compression_tokens += comp_tokens
            row_count += 1
        except (ValueError, IndexError):
            continue  # skip rows with missing or invalid token counts

if total_original_tokens > 0:
    overall_compression_ratio = round(total_compression_tokens / total_original_tokens, 4)
else:
    overall_compression_ratio = None

print(f"Total rows: {row_count}")
print(f"Total original tokens: {total_original_tokens}")
print(f"Total compression tokens: {total_compression_tokens}")
print(f"Overall compression ratio (column 5): {overall_compression_ratio}")

Total rows: 14779
Total original tokens: 236923
Total compression tokens: 177070
Overall compression ratio (column 5): 0.7474
